In [0]:
!git clone https://github.com/AkhilaMangipudi/PlanYourTravel.git

Cloning into 'PlanYourTravel'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 13 (delta 1), reused 10 (delta 1), pack-reused 0
Unpacking objects: 100% (13/13), done.


In [0]:
cd PlanYourTravel/

/content/PlanYourTravel


In [0]:
import numpy as np
import data.load

train_set, valid_set, dicts = data.load.atisfull()
w2idx, labels2idx = dicts['words2idx'], dicts['labels2idx']

train_x, _, train_label = train_set
val_x, _, val_label = valid_set

# Create index to word/label dicts
idx2w  = {w2idx[k]:k for k in w2idx}
idx2la = {labels2idx[k]:k for k in labels2idx}

# For conlleval script
words_train = [ list(map(lambda x: idx2w[x], w)) for w in train_x]
labels_train = [ list(map(lambda x: idx2la[x], y)) for y in train_label]
words_val = [ list(map(lambda x: idx2w[x], w)) for w in val_x]
labels_val = [ list(map(lambda x: idx2la[x], y)) for y in val_label]

n_classes = len(idx2la)
n_vocab = len(idx2w)

In [0]:
def replaceDigits(words):
  digits_list = ['DIGITDIGITDIGIT', 'DIGITDIGITDIGITDIGIT', 'DIGITDIGIT', 'DIGIT']
  words_train_new = []
  for sentence in words:
    sentence_new = [word if word not in digits_list else 'time' for word in sentence]
    words_train_new.append(sentence_new)
  return words_train_new

########### Replacing all possible digit combinations with a word 'time' ############
words_train = replaceDigits(words_train)
words_val = replaceDigits(words_val)

print(len(words_train))
print(len(words_val))
print(words_train[0])

4978
893
['i', 'want', 'to', 'fly', 'from', 'boston', 'at', 'time', 'am', 'and', 'arrive', 'in', 'denver', 'at', 'time', 'in', 'the', 'morning']


In [0]:
#Download Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-17 16:43:04--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-17 16:43:05--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-17 16:43:05--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove*.zip

In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.300d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
# pad documents to a max length of 4 words
from keras.preprocessing.sequence import pad_sequences
maxLen = 50
padded_docs = pad_sequences(train_x, maxlen=maxLen, padding='post')

Using TensorFlow backend.


In [0]:
#EMBEDDING_DIM =300
import re
embedding_matrix = np.zeros((len(dicts['words2idx']), 300))
for word, i in dicts['words2idx'].items():
  word = re.sub('[^\w\s]','',word) #remove punctuation
  #print(word)
  embedding_vector = embeddings_index.get(word)
  #print(embedding_vector)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

# **1D-CNN Models**

In [0]:
n_vocab = n_vocab + 1 #Adding an additional word padding

In [0]:
def get_past_window(data, label, window_size):
    cnn_input = []
    cnn_labels = []
    initial_pad = [n_vocab-1 for x in range(window_size-1)]
    for x in data:
        x= np.insert(x, 0, initial_pad)
        for i in range(window_size-1, len(x)):
            cnn_input.append(np.asarray(x[i-window_size+1:i+1]))

    for label in label:
        for temp in label:
            cnn_labels.append(temp)
    return np.asarray(cnn_input),  np.asarray(cnn_labels)           

In [0]:
def get_future_window(data, labels, window_size):
    cnn_input = []
    cnn_labels = []
    initial_pad = [n_vocab-1 for x in range(window_size-1)]
    for x in data:
        x = np.append(x, initial_pad)
        for i in range(0, len(x)-window_size+1):
            cnn_input.append(np.asarray(x[i:i+window_size]))

    for label in labels:
        for temp in label:
            cnn_labels.append(temp)
    return np.asarray(cnn_input),  np.asarray(cnn_labels)  

In [0]:
def get_past_and_future_window(data, label, window_size):
    cnn_input = []
    cnn_labels = []
    half_size = int(window_size/2);
    initial_pad = [n_vocab-1 for x in range(half_size)]
    for x in data:
        x = np.insert(x, 0, initial_pad)
        x = np.append(x, initial_pad)
        for i in range(half_size, len(x)-half_size):
            cnn_input.append(np.asarray(x[i-half_size:i+half_size+1]))

    for label in label:
        for temp in label:
            cnn_labels.append(temp)
    return np.asarray(cnn_input),  np.asarray(cnn_labels) 

In [0]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras import regularizers

from keras.layers import Conv1D

def get_conv_model(window_length, kernelsize=3):   
    model = Sequential()
    model.add(Embedding(n_vocab, 100, input_length=window_length))
    model.add(Conv1D(100, kernel_size=kernelsize, activation='relu', padding="same"))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()
    return model

In [0]:
#For prediction
from metrics.accuracy import conlleval

def print_evaluation(model, input_idx, output, input_words):
    y_pred = model.predict(input_idx)
    y_pred_temp = [np.argmax(t) for t in y_pred]

    labels_pred_val = []
    k = 0
    for x in input_words:
        temp = []
        for i in range(len(x)):
            temp.append(y_pred_temp[k])
            k = k + 1
        labels_pred_val.append(temp)
    
    labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
    con_dict = conlleval(labels_pred_val, labels_val, 
                                words_val, 'measure.txt')

    print('Precision = {}, Recall = {}, F1 = {}'.format(
                con_dict['r'], con_dict['p'], con_dict['f1']))

**Experiment with a window size of 3 using the past context words**

In [0]:
# past context of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 3, 100)            57300     
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 1, 100)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1, 100)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 100)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               12928     
_________________________________________________________________
dense_4 (Dense)              (None, 127)              

**Experiment with a window size of 5 using the past context words**

In [0]:
# past context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 5, 100)            57300     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 5, 100)            30100     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 2, 100)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 100)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_6 (Dense)              (None, 127)              

In [0]:
# past context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 5, 100)            57300     
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 2, 100)            0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 2, 100)            0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               25728     
_________________________________________________________________
dense_8 (Dense)              (None, 127)              

**Experiment with a window size of 7 using the past context words**

In [0]:
# past context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 7, 100)            57300     
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 3, 100)            0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 3, 100)            0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 128)               38528     
_________________________________________________________________
dense_10 (Dense)             (None, 127)              

In [0]:
# past context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 7, 100)            57300     
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 3, 100)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 3, 100)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_12 (Dense)             (None, 127)              

In [0]:
# past context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 7, 100)            57300     
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 3, 100)            0         
_________________________________________________________________
dropout_7 (Dropout)          (None, 3, 100)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 300)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_14 (Dense)             (None, 127)              

**Experiment with a window size of 9 using the past context words**

In [0]:
# past context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 9, 100)            57300     
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 4, 100)            0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 4, 100)            0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_16 (Dense)             (None, 127)              

In [0]:
# past context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 9, 100)            57300     
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_9 (MaxPooling1 (None, 4, 100)            0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 4, 100)            0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_18 (Dense)             (None, 127)              

In [0]:
# past context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = get_past_window(train_x, train_label, context_size)
val_input, val_labels = get_past_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_10 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_10 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_19 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_20 (Dense)             (None, 127)             

**Experiment with a window size of 3 using the future context words**

In [0]:
# future context of 3, kernel of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (None, 3, 100)            57300     
_________________________________________________________________
conv1d_16 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_16 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_16 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_16 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_31 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_32 (Dense)             (None, 127)             

**Experiment with a window size of 5 using the future context words**

In [0]:
# future context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 5, 100)            57300     
_________________________________________________________________
conv1d_17 (Conv1D)           (None, 5, 100)            30100     
_________________________________________________________________
max_pooling1d_17 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_17 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_17 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_33 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_34 (Dense)             (None, 127)             

In [0]:
# future context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (None, 5, 100)            57300     
_________________________________________________________________
conv1d_18 (Conv1D)           (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_18 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_18 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_18 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_35 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_36 (Dense)             (None, 127)             

**Experiment with a window size of 7 using the future context words**

In [0]:
# future context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_19 (Conv1D)           (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_19 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_19 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_19 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_37 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_38 (Dense)             (None, 127)             

In [0]:
# future context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_20 (Conv1D)           (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_20 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_20 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_20 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_39 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_40 (Dense)             (None, 127)             

In [0]:
# future context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_21 (Conv1D)           (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_21 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_21 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_41 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_42 (Dense)             (None, 127)             

**Experiment with a window size of 9 using the future context words**

In [0]:
# future context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_22 (Conv1D)           (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_22 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_22 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_44 (Dense)             (None, 127)             

In [0]:
# future context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_23 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_23 (Conv1D)           (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_23 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_23 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_23 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_45 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_46 (Dense)             (None, 127)             

In [0]:
# future context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = get_future_window(train_x, train_label, context_size)
val_input, val_labels = get_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_24 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_24 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_24 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_24 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_48 (Dense)             (None, 127)             

**Experiment with a window size of 3 using both future and past context words**

In [0]:
# past and future context of 3, kernel of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_25 (Embedding)     (None, 3, 100)            57300     
_________________________________________________________________
conv1d_25 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_25 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_49 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_50 (Dense)             (None, 127)             

**Experiment with a window size of 5 using both future and past context words**

In [0]:
# past and future context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (None, 3, 100)            57300     
_________________________________________________________________
conv1d_26 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_26 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_26 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_26 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_51 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_52 (Dense)             (None, 127)             

In [0]:
# past and future context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_27 (Embedding)     (None, 5, 100)            57300     
_________________________________________________________________
conv1d_27 (Conv1D)           (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_27 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_27 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_53 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_54 (Dense)             (None, 127)             

**Experiment with a window size of 7 using both future and past context words**

In [0]:
# past and future context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_28 (Conv1D)           (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_28 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_28 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_55 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_56 (Dense)             (None, 127)             

In [0]:
# past and future context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_29 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_29 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_29 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_29 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_57 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_58 (Dense)             (None, 127)             

In [0]:
# past and future context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (None, 7, 100)            57300     
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_30 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_30 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_30 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_60 (Dense)             (None, 127)             

**Experiment with a window size of 9 using both future and past context words**

In [0]:
# past and future context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_31"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_31 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_31 (Conv1D)           (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_31 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_31 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_31 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_61 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_62 (Dense)             (None, 127)             

In [0]:
# past and future context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_32 (Conv1D)           (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_32 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_32 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_32 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_63 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_64 (Dense)             (None, 127)             

In [0]:
# past and future context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = get_past_and_future_window(train_x, train_label, context_size)
val_input, val_labels = get_past_and_future_window(val_x, val_label, context_size)
model = get_conv_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "sequential_33"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_33 (Embedding)     (None, 9, 100)            57300     
_________________________________________________________________
conv1d_33 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_33 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_33 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_33 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_65 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_66 (Dense)             (None, 127)             

In [0]:
# Glove embedding

In [0]:
#Download Glove
!wget http://nlp.stanford.edu/data/glove.6B.zip

--2019-11-17 05:23:28--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2019-11-17 05:23:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2019-11-17 05:23:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
!unzip glove*.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = np.array([float(val) for val in values[1:]])
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [0]:
def preparePastCNNGloveInput(sentences_list, window_size, labels, glove_model):
  cnn_input = []
  cnn_labels = []
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence = ['padding'] * (window_size - 1) + sentence
    #Now for each word, take the (window_size - 1) previous words and itself, and form an embedding representation of each word
    for i in range(window_size - 1, len(sentence)):
      words_list = sentence[i-window_size+1:i+1]
      embedding = np.zeros(shape=(window_size,100))
      for j in range(window_size):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      cnn_input.append(embedding)

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return np.asarray(cnn_input), np.asarray(cnn_labels)

In [0]:
def prepareFutureCNNGloveInput(sentences_list, window_size, labels, glove_model):
  cnn_input = []
  cnn_labels = []
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence =  sentence + ['padding'] * (window_size - 1)
    #Now for each word, take the (window_size - 1) previous words and itself, and form an embedding representation of each word
    for i in range(0, len(sentence) - (window_size) + 1):
      words_list = sentence[i:i+window_size]
      embedding = np.zeros(shape=(window_size,100))
      for j in range(window_size):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      cnn_input.append(embedding)

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return np.asarray(cnn_input), np.asarray(cnn_labels)

In [0]:
def preparePastFutureCNNGloveInput(sentences_list, window_size, labels, glove_model):
  cnn_input = []
  cnn_labels = []
  half_size = int(window_size/2);
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence =  ['padding'] * (half_size) + sentence + ['padding'] * (half_size)
    #Now for each word, take the (window_size - 1) previous words and itself, and form an embedding representation of each word
    for i in range(half_size, len(sentence) - half_size):
      words_list = sentence[i-half_size:i+half_size+1]
      embedding = np.zeros(shape=(window_size,100))
      for j in range(window_size):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      cnn_input.append(embedding)

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return np.asarray(cnn_input), np.asarray(cnn_labels)

In [0]:
def replaceDigits(words):
  digits_list = ['DIGITDIGITDIGIT', 'DIGITDIGITDIGITDIGIT', 'DIGITDIGIT', 'DIGIT']
  words_train_new = []
  for sentence in words:
    sentence_new = [word if word not in digits_list else 'time' for word in sentence]
    words_train_new.append(sentence_new)
  return words_train_new

########### Replacing all possible digit combinations with a word 'time' ############
words_train = replaceDigits(words_train)
words_val = replaceDigits(words_val)

print(len(words_train))
print(len(words_val))
print(words_train[0])

4978
893
['i', 'want', 'to', 'fly', 'from', 'boston', 'at', 'time', 'am', 'and', 'arrive', 'in', 'denver', 'at', 'time', 'in', 'the', 'morning']


In [0]:
from keras import layers
from keras.models import Sequential, Model

def get_glove_model(window_size, kernelsize=3):
    input_tensor = layers.Input(shape=(window_size,100))
    x = Conv1D(100, kernel_size=kernelsize, activation='relu', padding="same") (input_tensor)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.25)(x)
    x = Flatten() (x)
    x = Dense(128, activation='relu')(x)
    x = Dense(n_classes, activation='softmax')(x)   
    optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    model = Model([input_tensor], x)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
    model.summary()
    return model

**Experiment with a window size of 3 using the past context words and glove embedding**

In [0]:
# past context of 3, kernel of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (None, 3, 100)            0         
_________________________________________________________________
conv1d_38 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_38 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_38 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_38 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_75 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_76 (Dense)             (None, 127)               1638

**Experiment with a window size of 5 using the past context words and glove embedding**

In [0]:
# past context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 5, 100)            0         
_________________________________________________________________
conv1d_39 (Conv1D)           (None, 5, 100)            30100     
_________________________________________________________________
max_pooling1d_39 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_39 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_39 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_77 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_78 (Dense)             (None, 127)               1638

In [0]:
# past context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 5, 100)            0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_40 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_40 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_79 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_80 (Dense)             (None, 127)               1638

**Experiment with a window size of 7 using the past context words and glove embedding**

In [0]:
# past context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 7, 100)            0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_41 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_41 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_41 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_82 (Dense)             (None, 127)               1638

In [0]:
# past context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 7, 100)            0         
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_42 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_42 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_42 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_83 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_84 (Dense)             (None, 127)               1638

In [0]:
# past context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_43 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_43 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_43 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_85 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_86 (Dense)             (None, 127)               1638

**Experiment with a window size of 9 using the past context words and glove embedding**

In [0]:
# past context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_44 (Conv1D)           (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_44 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_44 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_44 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_87 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_88 (Dense)             (None, 127)               163

In [0]:
# past context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_45 (Conv1D)           (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_45 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_45 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_45 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_89 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_90 (Dense)             (None, 127)               163

In [0]:
# past context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = preparePastCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_13 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_46 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_46 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_46 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_46 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_91 (Dense)             (None, 128)               51328     
_________________________________________________________________
dense_92 (Dense)             (None, 127)               163

**Experiment with a window size of 3 using the future context words and glove embedding**

In [0]:
# future context of 3, kernel of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        (None, 3, 100)            0         
_________________________________________________________________
conv1d_47 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_47 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_47 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_47 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_93 (Dense)             (None, 128)               12928     
_________________________________________________________________
dense_94 (Dense)             (None, 127)               163

**Experiment with a window size of 5 using the future context words and glove embedding**

In [0]:
# future context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_15 (InputLayer)        (None, 5, 100)            0         
_________________________________________________________________
conv1d_48 (Conv1D)           (None, 5, 100)            30100     
_________________________________________________________________
max_pooling1d_48 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_48 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_48 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_95 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_96 (Dense)             (None, 127)               163

In [0]:
# future context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_16 (InputLayer)        (None, 5, 100)            0         
_________________________________________________________________
conv1d_49 (Conv1D)           (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_49 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_49 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_49 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_97 (Dense)             (None, 128)               25728     
_________________________________________________________________
dense_98 (Dense)             (None, 127)               163

**Experiment with a window size of 7 using the future context words and glove embedding**

In [0]:
# future context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_17 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_50 (Conv1D)           (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_50 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_50 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_50 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_99 (Dense)             (None, 128)               38528     
_________________________________________________________________
dense_100 (Dense)            (None, 127)               163

In [0]:
# future context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_18 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_51 (Conv1D)           (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_51 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_51 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_101 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_102 (Dense)            (None, 127)               163

In [0]:
# future context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_19 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_52 (Conv1D)           (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_52 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_52 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_103 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_104 (Dense)            (None, 127)               163

**Experiment with a window size of 9 using the future context words and glove embedding**

In [0]:
# future context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_20 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_53 (Conv1D)           (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_53 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_53 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_105 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_106 (Dense)            (None, 127)               163

In [0]:
# future context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_54 (Conv1D)           (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_54 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_54 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_107 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_108 (Dense)            (None, 127)               163

In [0]:
# future context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = prepareFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = prepareFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_22 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_55 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_55 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_55 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_55 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_109 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_110 (Dense)            (None, 127)               163

**Experiment with a window size of 3 using both the future and past context words and glove embedding**

In [0]:
# past and future context of 3, kernel of 3
context_size = 3
kernel_size = 3
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_23 (InputLayer)        (None, 3, 100)            0         
_________________________________________________________________
conv1d_56 (Conv1D)           (None, 3, 100)            30100     
_________________________________________________________________
max_pooling1d_56 (MaxPooling (None, 1, 100)            0         
_________________________________________________________________
dropout_56 (Dropout)         (None, 1, 100)            0         
_________________________________________________________________
flatten_56 (Flatten)         (None, 100)               0         
_________________________________________________________________
dense_111 (Dense)            (None, 128)               12928     
_________________________________________________________________
dense_112 (Dense)            (None, 127)               163

**Experiment with a window size of 5 using both the future and past context words and glove embedding**

In [0]:
# past and future context of 5, kernel of 3
context_size = 5
kernel_size = 3
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_24 (InputLayer)        (None, 5, 100)            0         
_________________________________________________________________
conv1d_57 (Conv1D)           (None, 5, 100)            30100     
_________________________________________________________________
max_pooling1d_57 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_57 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_57 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_113 (Dense)            (None, 128)               25728     
_________________________________________________________________
dense_114 (Dense)            (None, 127)               163

In [0]:
# past and future context of 5, kernel of 5
context_size = 5
kernel_size = 5
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_25 (InputLayer)        (None, 5, 100)            0         
_________________________________________________________________
conv1d_58 (Conv1D)           (None, 5, 100)            50100     
_________________________________________________________________
max_pooling1d_58 (MaxPooling (None, 2, 100)            0         
_________________________________________________________________
dropout_58 (Dropout)         (None, 2, 100)            0         
_________________________________________________________________
flatten_58 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_115 (Dense)            (None, 128)               25728     
_________________________________________________________________
dense_116 (Dense)            (None, 127)               163

**Experiment with a window size of 7 using both the future and past context words and glove embedding**

In [0]:
# past and future context of 7, kernel of 3
context_size = 7
kernel_size = 3
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_26 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_59 (Conv1D)           (None, 7, 100)            30100     
_________________________________________________________________
max_pooling1d_59 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_59 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_59 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_117 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_118 (Dense)            (None, 127)               163

In [0]:
# past and future context of 7, kernel of 5
context_size = 7
kernel_size = 5
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_27 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_60 (Conv1D)           (None, 7, 100)            50100     
_________________________________________________________________
max_pooling1d_60 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_60 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_60 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_119 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_120 (Dense)            (None, 127)               163

In [0]:
# past and future context of 7, kernel of 7
context_size = 7
kernel_size = 7
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_28 (InputLayer)        (None, 7, 100)            0         
_________________________________________________________________
conv1d_61 (Conv1D)           (None, 7, 100)            70100     
_________________________________________________________________
max_pooling1d_61 (MaxPooling (None, 3, 100)            0         
_________________________________________________________________
dropout_61 (Dropout)         (None, 3, 100)            0         
_________________________________________________________________
flatten_61 (Flatten)         (None, 300)               0         
_________________________________________________________________
dense_121 (Dense)            (None, 128)               38528     
_________________________________________________________________
dense_122 (Dense)            (None, 127)               163

**Experiment with a window size of 9 using both the future and past context words and glove embedding**

In [0]:
# past and future context of 9, kernel of 3
context_size = 9
kernel_size = 3
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_29 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_62 (Conv1D)           (None, 9, 100)            30100     
_________________________________________________________________
max_pooling1d_62 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_62 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_62 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_123 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_124 (Dense)            (None, 127)               163

In [0]:
# past and future context of 9, kernel of 5
context_size = 9
kernel_size = 5
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_29"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_30 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_63 (Conv1D)           (None, 9, 100)            50100     
_________________________________________________________________
max_pooling1d_63 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_63 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_63 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_125 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_126 (Dense)            (None, 127)               163

In [0]:
# past and future context of 9, kernel of 7
context_size = 9
kernel_size = 7
cnn_input, cnn_labels = preparePastFutureCNNGloveInput(words_train, context_size, train_label, embeddings_index)
val_input, val_labels = preparePastFutureCNNGloveInput(words_val, context_size, val_label, embeddings_index)
model = get_glove_model(context_size, kernel_size)
model.fit(cnn_input, cnn_labels, epochs=30)
print_evaluation(model, val_input, val_labels, val_x)

Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        (None, 9, 100)            0         
_________________________________________________________________
conv1d_64 (Conv1D)           (None, 9, 100)            70100     
_________________________________________________________________
max_pooling1d_64 (MaxPooling (None, 4, 100)            0         
_________________________________________________________________
dropout_64 (Dropout)         (None, 4, 100)            0         
_________________________________________________________________
flatten_64 (Flatten)         (None, 400)               0         
_________________________________________________________________
dense_127 (Dense)            (None, 128)               51328     
_________________________________________________________________
dense_128 (Dense)            (None, 127)               163

# **2D-CNN Models**


In [0]:
glove_file = 'glove.6B.100d.txt'

In [0]:
#Load the embeddings file
import numpy as np
def loadGloveModel(gloveFile):
    print("Loading Glove Model")
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model

##################Loading the Glove Model into a dictionary#######################
glove_model = loadGloveModel(glove_file)

## **CNN - Past Context + GLoVE Model**

In [0]:
######################### For past CNN using Glove Embeddings ###############################
'''
Using glove Embeddings here
For any given word, take a window of n words, i.e. n-1 previous words and the current word
Also, to give weight to the current word for which the label is being predicted, take a context surrounding the current word
'''

import numpy as np

def preparePastCNNGloveInput(sentences_list, window_size, labels):
  cnn_input = []
  cnn_labels = []
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence = ['padding'] * (window_size - 1) + sentence
    #Now for each word, take the (window_size - 1) previous words and itself, and form an embedding representation of each word
    for i in range(window_size - 1, len(sentence)):
      words_list = sentence[i-window_size+1:i+1]
      embedding = np.zeros(shape=(window_size,100))
      for j in range(window_size):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      cnn_input.append(embedding)

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return cnn_input, cnn_labels

In [0]:
cnn_glove_train_input, cnn_glove_train_labels = preparePastCNNGloveInput(words_train, 5, train_label)
print("Training input length is {}".format(len(cnn_glove_train_input)))

cnn_glove_val_input, cnn_glove_val_labels = preparePastCNNGloveInput(words_val, 5, val_label)
print("Test input length is {}".format(len(cnn_glove_val_input)))

Training input length is 56590
Test input length is 9198


In [0]:

############################# For context inputs #####################################
'''
To emphasise on the word for which the slot is being predicted, add the surrounding context words again after the
max pooling layer to the CNN. 
'''
import numpy as np
#For any given word, consider a window of n words(previous) inclusing the current word as well
def preparePastCNNGloveContextInput(sentences_list, cs):
  context_input = []
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence = ['padding'] * cs + sentence + ['padding'] * cs
    for i in range(cs, len(sentence)-cs):
      words_list = sentence[i-cs:i+cs+1]
      embedding = np.zeros(shape=(2*cs+1,100))
      for j in range(2*cs+1):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      context_input.append(embedding)
  return context_input


In [0]:
context_glove_train_input = preparePastCNNGloveContextInput(words_train, 3)
context_glove_val_input = preparePastCNNGloveContextInput(words_val, 3)


In [0]:
x_train = np.asarray(cnn_glove_train_input)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
print("Shape of x_train", x_train.shape)
y_train = np.asarray(cnn_glove_train_labels)
print("Shape of y_train", y_train.shape)

x_val = np.asarray(cnn_glove_val_input)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], 1)
print("Shape of x_test", x_val.shape)
y_val = np.asarray(cnn_glove_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_glove_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], x_train_context.shape[2], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_glove_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], x_val_context.shape[2], 1)
print("Shape of x_test_context", x_val_context.shape)

Shape of x_train (56590, 5, 100, 1)
Shape of y_train (56590,)
Shape of x_test (9198, 5, 100, 1)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 100, 1)
Shape of x_test_context (9198, 7, 100, 1)


In [0]:
################ Model architecture ##############
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras import regularizers
from keras import layers

def sequentialCNNModel(window_size, cs):
  input_tensor = layers.Input(shape=(window_size,100,1))
  x = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(input_tensor)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Flatten()(x)

  input_context = layers.Input(shape=(2*cs+1,100,1))
  flatten_context = layers.Flatten()(input_context)

  concat = layers.concatenate([x, flatten_context], axis=-1)
  dense = layers.Dense(100, activation='relu')(concat)
  output = layers.Dense(n_classes, activation ='softmax')(dense)
  model = Model([input_tensor, input_context], output)
  optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
  model.summary()
  return model

In [0]:
past_cnn_model = sequentialCNNModel(5, 3)

Model: "model_5"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 5, 100, 1)    0                                            
__________________________________________________________________________________________________
conv2d_6 (Conv2D)               (None, 5, 100, 100)  1000        input_10[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_6 (MaxPooling2D)  (None, 2, 50, 100)   0           conv2d_6[0][0]                   
__________________________________________________________________________________________________
input_11 (InputLayer)           (None, 7, 100, 1)    0                                            
____________________________________________________________________________________________

In [0]:
past_cnn_model.fit([x_train, x_train_context], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 8s 143us/step - loss: 0.3405 - acc: 0.9280
Epoch 2/10
56590/56590 [==============================] - 8s 135us/step - loss: 0.0977 - acc: 0.9751
Epoch 3/10
56590/56590 [==============================] - 8s 135us/step - loss: 0.0636 - acc: 0.9824
Epoch 4/10
56590/56590 [==============================] - 8s 136us/step - loss: 0.0454 - acc: 0.9865
Epoch 5/10
56590/56590 [==============================] - 8s 135us/step - loss: 0.0359 - acc: 0.9889
Epoch 6/10
56590/56590 [==============================] - 8s 134us/step - loss: 0.0306 - acc: 0.9903
Epoch 7/10
56590/56590 [==============================] - 8s 133us/step - loss: 0.0251 - acc: 0.9925
Epoch 8/10
56590/56590 [==============================] - 7s 131us/step - loss: 0.0224 - acc: 0.9933
Epoch 9/10
56590/56590 [==============================] - 7s 130us/step - loss: 0.0208 - acc: 0.9934
Epoch 10/10
56590/56590 [==============================] - 8s 134us/step - loss: 0.0192 - a

In [0]:
#For prediction
from metrics.accuracy import conlleval

def reportResults(model, x_val, x_val_context):
  y_pred = model.predict([x_val, x_val_context])
  y_pred_temp = [np.argmax(t) for t in y_pred]

  labels_pred_val = []
  k = 0
  for x in val_x:
    temp = []
    for i in range(len(x)):
      temp.append(y_pred_temp[k])
      k = k + 1
    labels_pred_val.append(temp)

  labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
  con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

  print('Precision = {}, Recall = {}, F1 = {}'.format(
              con_dict['r'], con_dict['p'], con_dict['f1']))

In [0]:
print("Results for the past CNN")
reportResults(past_cnn_model, x_val, x_val_context)

Results for the past CNN
Precision = 91.26, Recall = 90.84, F1 = 91.05


## **CNN - Future Context + GLoVE Model**

In [0]:

######################### For future CNN ###############################

def prepareFutureCNNGloveInput(sentences_list, window_size, labels):
  cnn_input = []
  cnn_labels = []
  for sentence in sentences_list:
    #prepend a pad word to the beginning of the sentence
    sentence =  sentence + ['padding'] * (window_size - 1)
    #Now for each word, take the (window_size - 1) previous words and itself, and form an embedding representation of each word
    for i in range(0, len(sentence) - (window_size) + 1):
      words_list = sentence[i:i+window_size]
      embedding = np.zeros(shape=(window_size,100))
      for j in range(window_size):
        if words_list[j] in glove_model:
          embedding[j, :] = glove_model[words_list[j]]
      cnn_input.append(embedding)

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return cnn_input, cnn_labels

In [0]:
cnn_glove_train_input, cnn_glove_train_labels = prepareFutureCNNGloveInput(words_train, 5, train_label)
print("Training input length is {}".format(len(cnn_glove_train_input)))

cnn_glove_val_input, cnn_glove_val_labels = prepareFutureCNNGloveInput(words_val, 5, val_label)
print("Test input length is {}".format(len(cnn_glove_val_input)))

Training input length is 56590
Test input length is 9198


In [0]:
context_glove_train_input = preparePastCNNGloveContextInput(words_train, 3)
context_glove_val_input = preparePastCNNGloveContextInput(words_val, 3)

In [0]:
############## Preparing the Inputs ##############
x_train = np.asarray(cnn_glove_train_input)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], x_train.shape[2], 1)
print("Shape of x_train", x_train.shape)
y_train = np.asarray(cnn_glove_train_labels)
print("Shape of y_train", y_train.shape)

x_val = np.asarray(cnn_glove_val_input)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], x_val.shape[2], 1)
print("Shape of x_test", x_val.shape)
y_val = np.asarray(cnn_glove_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_glove_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], x_train_context.shape[2], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_glove_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], x_val_context.shape[2], 1)
print("Shape of x_test_context", x_val_context.shape)

Shape of x_train (56590, 5, 100, 1)
Shape of y_train (56590,)
Shape of x_test (9198, 5, 100, 1)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 100, 1)
Shape of x_test_context (9198, 7, 100, 1)


In [0]:
future_cnn_model = sequentialCNNModel(5, 3)

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 5, 100, 1)    0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 5, 100, 100)  1000        input_3[0][0]                    
__________________________________________________________________________________________________
max_pooling2d_2 (MaxPooling2D)  (None, 2, 50, 100)   0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 7, 100, 1)    0                                            
____________________________________________________________________________________________

In [0]:
future_cnn_model.fit([x_train, x_train_context], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 8s 141us/step - loss: 0.3087 - acc: 0.9315
Epoch 2/10
56590/56590 [==============================] - 8s 134us/step - loss: 0.1060 - acc: 0.9700
Epoch 3/10
56590/56590 [==============================] - 8s 134us/step - loss: 0.0762 - acc: 0.9767
Epoch 4/10
56590/56590 [==============================] - 8s 135us/step - loss: 0.0594 - acc: 0.9810
Epoch 5/10
56590/56590 [==============================] - 8s 134us/step - loss: 0.0490 - acc: 0.9842
Epoch 6/10
56590/56590 [==============================] - 8s 133us/step - loss: 0.0420 - acc: 0.9858
Epoch 7/10
56590/56590 [==============================] - 7s 130us/step - loss: 0.0372 - acc: 0.9878
Epoch 8/10
56590/56590 [==============================] - 7s 129us/step - loss: 0.0330 - acc: 0.9885
Epoch 9/10
56590/56590 [==============================] - 7s 132us/step - loss: 0.0290 - acc: 0.9902
Epoch 10/10
56590/56590 [==============================] - 7s 127us/step - loss: 0.0277 - a

In [0]:
print("Results for the future CNN")
reportResults(future_cnn_model, x_val, x_val_context)

Results for the future CNN
Precision = 90.76, Recall = 90.22, F1 = 90.49


## **CNN - Bidirectional Context + GLoVE Model**


In [0]:
############################### BiDirectional CNN #############################
################### Preparing the inputs #####################

cnn_past_train_input, cnn_past_train_labels = preparePastCNNGloveInput(words_train, 5, train_label)
print("Training input length is {}".format(len(cnn_past_train_input)))

cnn_past_val_input, cnn_past_val_labels = preparePastCNNGloveInput(words_val, 5, val_label)
print("Test input length is {}".format(len(cnn_past_val_input)))

cnn_future_train_input, cnn_future_train_labels = prepareFutureCNNGloveInput(words_train, 5, train_label)
print("Training input length is {}".format(len(cnn_future_train_input)))

cnn_future_val_input, cnn_future_val_labels = prepareFutureCNNGloveInput(words_val, 5, val_label)
print("Test input length is {}".format(len(cnn_future_val_input)))

context_train_input = preparePastCNNGloveContextInput(words_train, 3)
context_val_input = preparePastCNNGloveContextInput(words_val, 3)

x_train_past = np.asarray(cnn_past_train_input)
x_train_past = x_train_past.reshape(x_train_past.shape[0], x_train_past.shape[1], x_train_past.shape[2], 1)

x_train_future = np.asarray(cnn_future_train_input)
x_train_future = x_train_future.reshape(x_train_future.shape[0], x_train_future.shape[1], x_train_future.shape[2], 1)

y_train = np.asarray(cnn_future_train_labels)
print("Shape of y_train", y_train.shape)

x_val_past = np.asarray(cnn_past_val_input)
x_val_past = x_val_past.reshape(x_val_past.shape[0], x_val_past.shape[1], x_val_past.shape[2], 1)

x_val_future = np.asarray(cnn_future_val_input)
x_val_future = x_val_future.reshape(x_val_future.shape[0], x_val_future.shape[1], x_val_future.shape[2], 1)

y_val = np.asarray(cnn_future_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], x_train_context.shape[2], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], x_val_context.shape[2], 1)
print("Shape of x_test_context", x_val_context.shape)

Training input length is 56590
Test input length is 9198
Training input length is 56590
Test input length is 9198
Shape of y_train (56590,)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 100, 1)
Shape of x_test_context (9198, 7, 100, 1)


In [0]:
################ Model architecture ##############
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras import regularizers
from keras import layers

def biSequentialCNNModel(window_size, cs):
  input_tensor_past = layers.Input(shape=(window_size,100,1))
  x = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(input_tensor_past)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Flatten()(x)

  input_context = layers.Input(shape=(2*cs+1,100,1))
  flatten_context = layers.Flatten()(input_context)

  input_tensor_future = layers.Input(shape=(window_size,100,1))
  x_future = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(input_tensor_future)
  x_future = layers.MaxPooling2D(pool_size=(2,2))(x_future)
  x_future = layers.Flatten()(x_future)


  concat_past = layers.concatenate([x, flatten_context], axis=-1)
  concat_future = layers.concatenate([x_future, flatten_context], axis = -1)

  concat = layers.concatenate([concat_past, concat_future], axis = -1)
  dense = layers.Dense(100, activation='relu')(concat)
  output = layers.Dense(n_classes, activation ='softmax')(dense)
  model = Model([input_tensor_past, input_context, input_tensor_future], output)
  optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
  model.summary()
  return model

In [0]:
bidirectional_cnn = biSequentialCNNModel(5, 3)

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 5, 100, 1)    0                                            
__________________________________________________________________________________________________
input_14 (InputLayer)           (None, 5, 100, 1)    0                                            
__________________________________________________________________________________________________
conv2d_7 (Conv2D)               (None, 5, 100, 100)  1000        input_12[0][0]                   
__________________________________________________________________________________________________
conv2d_8 (Conv2D)               (None, 5, 100, 100)  1000        input_14[0][0]                   
____________________________________________________________________________________________

In [0]:
bidirectional_cnn.fit([x_train_past, x_train_context, x_train_future], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 11s 186us/step - loss: 0.2891 - acc: 0.9372
Epoch 2/10
56590/56590 [==============================] - 10s 177us/step - loss: 0.0903 - acc: 0.9756
Epoch 3/10
56590/56590 [==============================] - 10s 176us/step - loss: 0.0600 - acc: 0.9826
Epoch 4/10
56590/56590 [==============================] - 10s 175us/step - loss: 0.0440 - acc: 0.9862
Epoch 5/10
56590/56590 [==============================] - 10s 178us/step - loss: 0.0346 - acc: 0.9889
Epoch 6/10
56590/56590 [==============================] - 10s 176us/step - loss: 0.0313 - acc: 0.9899
Epoch 7/10
56590/56590 [==============================] - 10s 176us/step - loss: 0.0250 - acc: 0.9920
Epoch 8/10
56590/56590 [==============================] - 10s 176us/step - loss: 0.0240 - acc: 0.9920
Epoch 9/10
56590/56590 [==============================] - 10s 177us/step - loss: 0.0222 - acc: 0.9929
Epoch 10/10
56590/56590 [==============================] - 10s 176us/step - loss: 

In [0]:
def reportBiResults(model, x_val_past, x_val_context, x_val_future):
  y_pred = model.predict([x_val_past, x_val_context, x_val_future])
  y_pred_temp = [np.argmax(t) for t in y_pred]

  labels_pred_val = []
  k = 0
  for x in val_x:
    temp = []
    for i in range(len(x)):
      temp.append(y_pred_temp[k])
      k = k + 1
    labels_pred_val.append(temp)

  labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
  con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

  print('Precision = {}, Recall = {}, F1 = {}'.format(
              con_dict['r'], con_dict['p'], con_dict['f1']))

In [0]:
print("Results for the bidirectional CNN")
reportBiResults(bidirectional_cnn, x_val_past, x_val_context, x_val_future)

Results for the bidirectional CNN
Precision = 92.0, Recall = 90.28, F1 = 91.13


## **Sequential CNN - Past Context Model**

In [0]:
###################### Sequential CNN using a Keras Embedding Layer ################

In [0]:
print(n_vocab)
n_vocab = n_vocab + 1
print(n_vocab)

572
573


In [0]:
######################## Past CNN ##################

def preparePastCNNInput(encoded_list, window_size, labels):
  cnn_input = []
  cnn_labels = []
  for x in encoded_list:
    x= np.insert(x, 0, [n_vocab-1] * (window_size - 1))
    for i in range(window_size - 1, len(x)):
      cnn_input.append(np.asarray(x[i-window_size+1:i+1]))

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return cnn_input, cnn_labels

In [0]:
past_cnn_train_input, past_cnn_train_labels = preparePastCNNInput(train_x, 5, train_label)
past_cnn_val_input, past_cnn_val_labels = preparePastCNNInput(val_x, 5, val_label)

print("Training input size: ", len(past_cnn_train_input))
print("Testing input size: ", len(past_cnn_val_input))

print("Training input labels size: ", len(past_cnn_train_labels))
print("Testing input labels size: ", len(past_cnn_val_labels))

Training input size:  56590
Testing input size:  9198
Training input labels size:  56590
Testing input labels size:  9198


In [0]:
##################### Context input ###################
def prepareCNNContextInput(encoded_list, cs):
  context_input = []
  for x in encoded_list:
    x= np.insert(x, 0, [n_vocab-1] * (cs))
    x = np.append(x, [n_vocab - 1] * cs)
    for i in range(cs, len(x)-cs):
      context_input.append(np.asarray(x[i-cs:i+cs+1]))
  return context_input

In [0]:
context_train_input = prepareCNNContextInput(train_x, 3)
context_val_input = prepareCNNContextInput(val_x, 3)

print("Context input train length: ", len(context_train_input))
print("Context input test length: ", len(context_val_input))

Context input train length:  56590
Context input test length:  9198


In [0]:
x_train = np.asarray(past_cnn_train_input)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
print("Shape of x_train", x_train.shape)
y_train = np.asarray(past_cnn_train_labels)
print("Shape of y_train", y_train.shape)

x_val = np.asarray(past_cnn_val_input)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)
print("Shape of x_test", x_val.shape)
y_val = np.asarray(past_cnn_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], 1)
print("Shape of x_test_context", x_val_context.shape)

Shape of x_train (56590, 5, 1)
Shape of y_train (56590,)
Shape of x_test (9198, 5, 1)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 1)
Shape of x_test_context (9198, 7, 1)


In [0]:
################# Model architecture #######################
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras import regularizers
from keras import layers

def sequentialCNNModel(window_size, cs):
  input_tensor = layers.Input(shape=(window_size,1))
  x = layers.Embedding(n_vocab,100)(input_tensor)
  x = layers.Reshape((window_size, 100, 1))(x)
  x = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(x)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Flatten()(x)

  input_context = layers.Input(shape=(2*cs + 1,1))
  context_embedding = layers.Embedding(n_vocab, 100)(input_context)
  flatten_context = layers.Flatten()(context_embedding)

  concat = layers.concatenate([x, flatten_context], axis=-1)
  dense = layers.Dense(100, activation='relu')(concat)
  output = layers.Dense(n_classes, activation ='softmax')(dense)
  model = Model([input_tensor, input_context], output)
  optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
  model.summary()
  return model

In [0]:
past_cnn_model = sequentialCNNModel(5,3)

Model: "model_10"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_22 (InputLayer)           (None, 5, 1)         0                                            
__________________________________________________________________________________________________
embedding_8 (Embedding)         (None, 5, 1, 100)    57300       input_22[0][0]                   
__________________________________________________________________________________________________
reshape_5 (Reshape)             (None, 5, 100, 1)    0           embedding_8[0][0]                
__________________________________________________________________________________________________
conv2d_13 (Conv2D)              (None, 5, 100, 100)  1000        reshape_5[0][0]                  
___________________________________________________________________________________________

In [0]:
past_cnn_model.fit([x_train, x_train_context], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 10s 168us/step - loss: 0.3973 - acc: 0.9199
Epoch 2/10
56590/56590 [==============================] - 9s 151us/step - loss: 0.0735 - acc: 0.9823
Epoch 3/10
56590/56590 [==============================] - 9s 152us/step - loss: 0.0442 - acc: 0.9883
Epoch 4/10
56590/56590 [==============================] - 9s 152us/step - loss: 0.0298 - acc: 0.9915
Epoch 5/10
56590/56590 [==============================] - 9s 152us/step - loss: 0.0214 - acc: 0.9937
Epoch 6/10
56590/56590 [==============================] - 9s 154us/step - loss: 0.0166 - acc: 0.9950
Epoch 7/10
56590/56590 [==============================] - 9s 152us/step - loss: 0.0138 - acc: 0.9957
Epoch 8/10
56590/56590 [==============================] - 9s 153us/step - loss: 0.0118 - acc: 0.9964
Epoch 9/10
56590/56590 [==============================] - 9s 152us/step - loss: 0.0103 - acc: 0.9969
Epoch 10/10
56590/56590 [==============================] - 9s 151us/step - loss: 0.0088 - 

In [0]:
#For prediction
from metrics.accuracy import conlleval

def reportResults(model, x_val, x_val_context):
  y_pred = model.predict([x_val, x_val_context])
  y_pred_temp = [np.argmax(t) for t in y_pred]

  labels_pred_val = []
  k = 0
  for x in val_x:
    temp = []
    for i in range(len(x)):
      temp.append(y_pred_temp[k])
      k = k + 1
    labels_pred_val.append(temp)

  labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
  con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

  print('Precision = {}, Recall = {}, F1 = {}'.format(
              con_dict['r'], con_dict['p'], con_dict['f1']))

In [0]:
print("Results for Past CNN ")
print(reportResults(past_cnn_model, x_val, x_val_context))

Results for Past CNN 
Precision = 91.54, Recall = 90.84, F1 = 91.19
None


## **Sequential CNN - Future Context Model**


In [0]:
############## Future CNN ###################

def prepareFutureCNNInput(encoded_list, window_size, labels):
  cnn_input = []
  cnn_labels = []
  for x in encoded_list:
    x= np.append(x, [n_vocab-1] * (window_size - 1))
    for i in range(0, len(x) - window_size + 1):
      cnn_input.append(np.asarray(x[i:i+window_size]))

  for label in labels:
    for temp in label:
      cnn_labels.append(temp)
  return cnn_input, cnn_labels

In [0]:
future_cnn_train_input, future_cnn_train_labels = prepareFutureCNNInput(train_x, 5, train_label)
future_cnn_val_input, future_cnn_val_labels = prepareFutureCNNInput(val_x, 5, val_label)

print("Training input length: ", len(future_cnn_train_input))
print("Testing input length: ", len(future_cnn_val_input))

Training input length:  56590
Testing input length:  9198


In [0]:
context_train_input = prepareCNNContextInput(train_x, 3)
context_val_input = prepareCNNContextInput(val_x, 3)

print("Context input train length: ", len(context_train_input))
print("Context input test length: ", len(context_val_input))

Context input train length:  56590
Context input test length:  9198


In [0]:
x_train = np.asarray(future_cnn_train_input)
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1], 1)
print("Shape of x_train", x_train.shape)
y_train = np.asarray(future_cnn_train_labels)
print("Shape of y_train", y_train.shape)

x_val = np.asarray(future_cnn_val_input)
x_val = x_val.reshape(x_val.shape[0], x_val.shape[1], 1)
print("Shape of x_test", x_val.shape)
y_val = np.asarray(future_cnn_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], 1)
print("Shape of x_test_context", x_val_context.shape)

Shape of x_train (56590, 5, 1)
Shape of y_train (56590,)
Shape of x_test (9198, 5, 1)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 1)
Shape of x_test_context (9198, 7, 1)


In [0]:
future_cnn_model = sequentialCNNModel(5, 3)

Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_17 (InputLayer)           (None, 5, 1)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 5, 1, 100)    57300       input_17[0][0]                   
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 5, 100, 1)    0           embedding_3[0][0]                
__________________________________________________________________________________________________
conv2d_10 (Conv2D)              (None, 5, 100, 100)  1000        reshape_2[0][0]                  
____________________________________________________________________________________________

In [0]:
future_cnn_model.fit([x_train, x_train_context], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 9s 163us/step - loss: 0.4069 - acc: 0.9134
Epoch 2/10
56590/56590 [==============================] - 8s 149us/step - loss: 0.0801 - acc: 0.9793
Epoch 3/10
56590/56590 [==============================] - 8s 149us/step - loss: 0.0502 - acc: 0.9854
Epoch 4/10
56590/56590 [==============================] - 9s 150us/step - loss: 0.0361 - acc: 0.9889
Epoch 5/10
56590/56590 [==============================] - 8s 149us/step - loss: 0.0293 - acc: 0.9904
Epoch 6/10
56590/56590 [==============================] - 9s 151us/step - loss: 0.0229 - acc: 0.9928
Epoch 7/10
56590/56590 [==============================] - 9s 151us/step - loss: 0.0208 - acc: 0.9932
Epoch 8/10
56590/56590 [==============================] - 9s 151us/step - loss: 0.0173 - acc: 0.9943
Epoch 9/10
56590/56590 [==============================] - 8s 148us/step - loss: 0.0148 - acc: 0.9952
Epoch 10/10
56590/56590 [==============================] - 9s 150us/step - loss: 0.0132 - a

In [0]:
print("Results for Future CNN ")
print(reportResults(future_cnn_model, x_val, x_val_context))

Results for Future CNN 
Precision = 91.68, Recall = 91.65, F1 = 91.67
None


## **Sequential CNN - Bidirectional Context Model**


In [0]:
######################## BiDirectional CNN #######################
################### Inputs Preparation ########################

cnn_past_train_input, cnn_past_train_labels = preparePastCNNInput(train_x, 5, train_label)
print("Training input length is {}".format(len(cnn_past_train_input)))

cnn_past_val_input, cnn_past_val_labels = preparePastCNNInput(val_x, 5, val_label)
print("Test input length is {}".format(len(cnn_past_val_input)))

cnn_future_train_input, cnn_future_train_labels = prepareFutureCNNInput(train_x, 5, train_label)
print("Training input length is {}".format(len(cnn_future_train_input)))

cnn_future_val_input, cnn_future_val_labels = prepareFutureCNNInput(val_x, 5, val_label)
print("Test input length is {}".format(len(cnn_future_val_input)))

context_train_input = prepareCNNContextInput(train_x, 3)
context_val_input = prepareCNNContextInput(val_x, 3)

x_train_past = np.asarray(cnn_past_train_input)
x_train_past = x_train_past.reshape(x_train_past.shape[0], x_train_past.shape[1], 1)

x_train_future = np.asarray(cnn_future_train_input)
x_train_future = x_train_future.reshape(x_train_future.shape[0], x_train_future.shape[1], 1)

y_train = np.asarray(cnn_future_train_labels)
print("Shape of y_train", y_train.shape)

x_val_past = np.asarray(cnn_past_val_input)
x_val_past = x_val_past.reshape(x_val_past.shape[0], x_val_past.shape[1], 1)

x_val_future = np.asarray(cnn_future_val_input)
x_val_future = x_val_future.reshape(x_val_future.shape[0], x_val_future.shape[1], 1)

y_val = np.asarray(cnn_future_val_labels)
print("Shape of y_test", y_val.shape)


x_train_context = np.asarray(context_train_input)
x_train_context = x_train_context.reshape(x_train_context.shape[0], x_train_context.shape[1], 1)
print("Shape of x_train_context", x_train_context.shape)

x_val_context = np.asarray(context_val_input)
x_val_context = x_val_context.reshape(x_val_context.shape[0], x_val_context.shape[1], 1)
print("Shape of x_test_context", x_val_context.shape)

Training input length is 56590
Test input length is 9198
Training input length is 56590
Test input length is 9198
Shape of y_train (56590,)
Shape of y_test (9198,)
Shape of x_train_context (56590, 7, 1)
Shape of x_test_context (9198, 7, 1)


In [0]:
################ Model architecture ##############
from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Convolution1D, MaxPooling1D
from keras.utils import np_utils
from keras.layers.embeddings import Embedding
from keras.optimizers import *
from keras import regularizers
from keras import layers

def biSequentialCNNModel(window_size, cs):
  input_tensor_past = layers.Input(shape=(window_size, 1))
  x = layers.Embedding(n_vocab, 100)(input_tensor_past)
  x = layers.Reshape((window_size, 100, 1))(x)
  x = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(x)
  x = layers.MaxPooling2D(pool_size=(2,2))(x)
  x = layers.Flatten()(x)

  input_tensor_future = layers.Input(shape=(window_size, 1))
  x_future = layers.Embedding(n_vocab, 100)(input_tensor_future)
  x_future = layers.Reshape((window_size, 100, 1))(x_future)
  x_future = layers.Convolution2D(100, kernel_size=3, strides=1, activation='relu', padding='same')(x_future)
  x_future = layers.MaxPooling2D(pool_size=(2,2))(x_future)
  x_future = layers.Flatten()(x_future)

  input_context = layers.Input(shape=(2*cs + 1,1))
  context_embedding = layers.Embedding(n_vocab, 100)(input_context)
  flatten_context = layers.Flatten()(context_embedding)

  concat_past = layers.concatenate([x, flatten_context], axis=-1)
  concat_future = layers.concatenate([x_future, flatten_context], axis = -1)

  concat = layers.concatenate([concat_past, concat_future], axis = -1)
  dense = layers.Dense(100, activation='relu')(concat)
  output = layers.Dense(n_classes, activation ='softmax')(dense)
  model = Model([input_tensor_past, input_context, input_tensor_future], output)
  optim = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
  model.compile(loss='sparse_categorical_crossentropy', optimizer=optim, metrics=['accuracy'])
  model.summary()
  return model

In [0]:
cnn_bidirectional_model = biSequentialCNNModel(5, 3)

Model: "model_9"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           (None, 5, 1)         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           (None, 5, 1)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 5, 1, 100)    57300       input_19[0][0]                   
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, 5, 1, 100)    57300       input_20[0][0]                   
____________________________________________________________________________________________

In [0]:
cnn_bidirectional_model.fit([x_train_past, x_train_context, x_train_future], y_train, epochs=10)

Epoch 1/10
56590/56590 [==============================] - 12s 215us/step - loss: 0.3238 - acc: 0.9343
Epoch 2/10
56590/56590 [==============================] - 11s 201us/step - loss: 0.0600 - acc: 0.9852
Epoch 3/10
56590/56590 [==============================] - 11s 195us/step - loss: 0.0353 - acc: 0.9898
Epoch 4/10
56590/56590 [==============================] - 11s 195us/step - loss: 0.0262 - acc: 0.9921
Epoch 5/10
56590/56590 [==============================] - 11s 195us/step - loss: 0.0188 - acc: 0.9942
Epoch 6/10
56590/56590 [==============================] - 11s 196us/step - loss: 0.0146 - acc: 0.9957
Epoch 7/10
56590/56590 [==============================] - 11s 198us/step - loss: 0.0133 - acc: 0.9962
Epoch 8/10
56590/56590 [==============================] - 11s 197us/step - loss: 0.0122 - acc: 0.9963
Epoch 9/10
56590/56590 [==============================] - 11s 196us/step - loss: 0.0098 - acc: 0.9970
Epoch 10/10
56590/56590 [==============================] - 11s 197us/step - loss: 

In [0]:
def reportBiResults(model, x_val_past, x_val_context, x_val_future):
  y_pred = model.predict([x_val_past, x_val_context, x_val_future])
  y_pred_temp = [np.argmax(t) for t in y_pred]

  labels_pred_val = []
  k = 0
  for x in val_x:
    temp = []
    for i in range(len(x)):
      temp.append(y_pred_temp[k])
      k = k + 1
    labels_pred_val.append(temp)

  labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
  con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

  print('Precision = {}, Recall = {}, F1 = {}'.format(
              con_dict['r'], con_dict['p'], con_dict['f1']))

In [0]:
print("Results for bidirectional CNN")
print(reportBiResults(cnn_bidirectional_model, x_val_past, x_val_context, x_val_future))

Results for bidirectional CNN
Precision = 93.02, Recall = 92.47, F1 = 92.74
None


# **RNN  Models**

## **SimpleRNN Model**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D

model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Dropout(0.25))
model.add(SimpleRNN(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()

Using TensorFlow backend.






Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 100)         57200     
_________________________________________________________________
dropout_1 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, None, 100)         20100     
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 127)         12827     
Total params: 90,127
Trainable params: 90,127
Non-trainable params: 0
_________________________________________________________________


In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    
    bar = progressbar.ProgressBar(max_value=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










100% (4978 of 4978) |####################| Elapsed Time: 0:00:43 Time:  0:00:43
  0% (14 of 4978) |                      | Elapsed Time: 0:00:00 ETA:   0:00:36

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:00:37 Time:  0:00:37
  0% (15 of 4978) |                      | Elapsed Time: 0:00:00 ETA:   0:00:34

...

100% (4978 of 4978) |####################| Elapsed Time: 0:00:35 Time:  0:00:35
  0% (14 of 4978) |                      | Elapsed Time: 0:00:00 ETA:   0:00:35

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:00:35 Time:  0:00:35
  0% (15 of 4978) |                      | Elapsed Time: 0:00:00 ETA:   0:00:33

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:00:36 Time:  0:00:36


In [0]:
from metrics.accuracy import conlleval

labels_pred_val = []

bar = progressbar.ProgressBar(max_value=len(val_x))
for n_batch, sent in bar(enumerate(val_x)):
    label = val_label[n_batch]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))


100% (893 of 893) |######################| Elapsed Time: 0:00:01 Time:  0:00:01


Precision = 92.25, Recall = 92.7, F1 = 92.47


## **SimpleRNN + GLoVE Models**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D

model = Sequential()
model.add(Embedding(len(dicts['words2idx']), 300, weights=[embedding_matrix], input_length=maxLen, trainable=False))
model.add(Dropout(0.25))
model.add(SimpleRNN(300,return_sequences=True))
model.add(SimpleRNN(200,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 300)           171600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 300)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 46, 300)           180300    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 46, 200)           100200    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 46, 127)           25527     
Total params: 477,627
Trainable params: 306,027
Non-trainable params: 171,600
_______________________________________________

In [0]:
import progressbar
n_epochs = 30
for i in range(n_epochs):
    print("Training epoch {}".format(i))
    bar = progressbar.ProgressBar(max_value=len(padded_docs))
    for n_batch, sent in bar(enumerate(padded_docs)):
        #print(sent.shape)
        label = train_label[n_batch]    
        if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
        else:
          label= label[-maxLen:]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        if sent.shape[1] > 1: #ignore 1 word sentences
          model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




100% (4978 of 4978) |####################| Elapsed Time: 0:02:12 Time:  0:02:12
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:14

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:02:11 Time:  0:02:11
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:11

...

100% (4978 of 4978) |####################| Elapsed Time: 0:02:14 Time:  0:02:14
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:39

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:02:13 Time:  0:02:13
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:16

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:02:12 Time:  0:02:12


In [0]:
#2 Simple RNNs, Glove, padded to maxLen, DIGIT handled, embedding dim =300
from metrics.accuracy import conlleval

padded_docs_val = pad_sequences(val_x, maxlen=maxLen, padding='post')
#print(padded_docs_val)

labels_pred_val = []
bar = progressbar.ProgressBar(max_value=len(padded_docs_val))
for n_batch, sent in bar(enumerate(padded_docs_val)):
    label = val_label[n_batch]
    if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
    else:
      label= label[-maxLen:]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))

100% (893 of 893) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


Precision = 90.17, Recall = 90.07, F1 = 90.12


In [0]:
#3 Two Layers of Simple RNNs, Glove, padded to maxLen, DIGIT handled, embedding dim =300

from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D

model = Sequential()
model.add(Embedding(len(dicts['words2idx']), 300, weights=[embedding_matrix], input_length=maxLen, trainable=False))
model.add(Dropout(0.25))
model.add(SimpleRNN(300,return_sequences=True))
model.add(SimpleRNN(200,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()











Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 46, 300)           171600    
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 300)           0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 46, 300)           180300    
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 46, 200)           100200    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 46, 127)           25527     
Total params: 477,627
Trainable params: 306,027
Non-trainable params: 171,600
_______________________________________________

In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    bar = progressbar.ProgressBar(max_value=len(padded_docs))
    for n_batch, sent in bar(enumerate(padded_docs)):
        #print(sent.shape)
        label = train_label[n_batch]    
        if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
        else:
          label= label[-maxLen:]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        if sent.shape[1] > 1: #ignore 1 word sentences
          model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where




100% (4978 of 4978) |####################| Elapsed Time: 0:02:12 Time:  0:02:12
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:14

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:02:11 Time:  0:02:11
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:11

...

100% (4978 of 4978) |####################| Elapsed Time: 0:02:14 Time:  0:02:14
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:39

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:02:13 Time:  0:02:13
  0% (4 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:02:16

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:02:12 Time:  0:02:12


In [0]:
#2 Simple RNNs, Glove, padded to maxLen, DIGIT handled, embedding dim =300
from metrics.accuracy import conlleval

padded_docs_val = pad_sequences(val_x, maxlen=maxLen, padding='post')
#print(padded_docs_val)

labels_pred_val = []
bar = progressbar.ProgressBar(max_value=len(padded_docs_val))
for n_batch, sent in bar(enumerate(padded_docs_val)):
    label = val_label[n_batch]
    if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
    else:
      label= label[-maxLen:]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))

100% (893 of 893) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


Precision = 90.17, Recall = 90.07, F1 = 90.12


## **LSTM Model**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D

model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Dropout(0.25))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()


Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, None, 100)         57200     
_________________________________________________________________
dropout_7 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
lstm_5 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
lstm_6 (LSTM)                (None, None, 100)         80400     
_________________________________________________________________
time_distributed_5 (TimeDist (None, None, 127)         12827     
Total params: 230,827
Trainable params: 230,827
Non-trainable params: 0
_________________________________________________________________


In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    bar = progressbar.ProgressBar(max_value=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where










100% (4978 of 4978) |####################| Elapsed Time: 0:01:33 Time:  0:01:33
  0% (6 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:34

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:01:28 Time:  0:01:28
  0% (6 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:30

...

100% (4978 of 4978) |####################| Elapsed Time: 0:01:25 Time:  0:01:25
  0% (6 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:30

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:01:24 Time:  0:01:24
  0% (6 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:01:27

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:01:24 Time:  0:01:24


In [0]:
from metrics.accuracy import conlleval

labels_pred_val = []

bar = progressbar.ProgressBar(max_value=len(val_x))
for n_batch, sent in bar(enumerate(val_x)):
    label = val_label[n_batch]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))


100% (893 of 893) |######################| Elapsed Time: 0:00:02 Time:  0:00:02


Precision = 92.07, Recall = 92.62, F1 = 92.35


## **LSTM + GLoVE Model**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, Bidirectional

model = Sequential()
model.add(Embedding(len(dicts['words2idx']), 300, weights=[embedding_matrix], input_length=maxLen, trainable=False))
model.add(Dropout(0.25))
model.add(LSTM(100,return_sequences=True))
model.add(LSTM(100,return_sequences=True))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 46, 300)           171600    
_________________________________________________________________
dropout_4 (Dropout)          (None, 46, 300)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 46, 100)           160400    
_________________________________________________________________
lstm_4 (LSTM)                (None, 46, 100)           80400     
_________________________________________________________________
time_distributed_4 (TimeDist (None, 46, 127)           12827     
Total params: 425,227
Trainable params: 253,627
Non-trainable params: 171,600
_________________________________________________________________


In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    bar = progressbar.ProgressBar(max_value=len(padded_docs))
    for n_batch, sent in bar(enumerate(padded_docs)):
        #print(sent.shape)
        label = train_label[n_batch]    
        if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
        else:
          label= label[-maxLen:]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        if sent.shape[1] > 1: #ignore 1 word sentences
          model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0


100% (4978 of 4978) |####################| Elapsed Time: 0:05:11 Time:  0:05:11
  0% (2 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:05:05

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:05:09 Time:  0:05:09
  0% (2 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:05:18

...

100% (4978 of 4978) |####################| Elapsed Time: 0:05:08 Time:  0:05:08
  0% (2 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:05:11

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:05:08 Time:  0:05:08
  0% (2 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:05:13

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:05:08 Time:  0:05:08


In [0]:
#Glove, padded to maxLen, DIGIT handled, embedding dim =300
from metrics.accuracy import conlleval

padded_docs_val = pad_sequences(val_x, maxlen=maxLen, padding='post')
#print(padded_docs_val)

labels_pred_val = []
bar = progressbar.ProgressBar(max_value=len(padded_docs_val))
for n_batch, sent in bar(enumerate(padded_docs_val)):
    label = val_label[n_batch]
    if len(label)<maxLen:     
          label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
    else:
      label= label[-maxLen:]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))

100% (893 of 893) |######################| Elapsed Time: 0:00:07 Time:  0:00:07


Precision = 93.34, Recall = 92.78, F1 = 93.06


## **Bidirectional LSTM Model**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, Bidirectional

model = Sequential()
model.add(Embedding(n_vocab,100))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Dropout(0.25))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, None, 100)         57200     
_________________________________________________________________
bidirectional_5 (Bidirection (None, None, 200)         160800    
_________________________________________________________________
bidirectional_6 (Bidirection (None, None, 200)         240800    
_________________________________________________________________
dropout_3 (Dropout)          (None, None, 200)         0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 127)         25527     
Total params: 484,327
Trainable params: 484,327
Non-trainable params: 0
_________________________________________________________________


In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    
    bar = progressbar.ProgressBar(max_value=len(train_x))
    for n_batch, sent in bar(enumerate(train_x)):
        label = train_label[n_batch]
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:] 
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0


100% (4978 of 4978) |####################| Elapsed Time: 0:03:18 Time:  0:03:18
  0% (3 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:03:34

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:03:10 Time:  0:03:10
  0% (3 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:03:50

...

100% (4978 of 4978) |####################| Elapsed Time: 0:03:12 Time:  0:03:12
  0% (3 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:03:43

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:03:13 Time:  0:03:13
  0% (3 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:03:52

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:03:14 Time:  0:03:14


In [0]:
from metrics.accuracy import conlleval

labels_pred_val = []

bar = progressbar.ProgressBar(max_value=len(val_x))
for n_batch, sent in bar(enumerate(val_x)):
    label = val_label[n_batch]
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))



100% (893 of 893) |######################| Elapsed Time: 0:00:04 Time:  0:00:04


Precision = 95.17, Recall = 95.81, F1 = 95.49


## **Bidirectional LSTM + GLoVE Model**

In [0]:
from keras.models import Sequential
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.core import Dense, Dropout
from keras.layers.wrappers import TimeDistributed
from keras.layers import Convolution1D, Bidirectional

model = Sequential()
model.add(Embedding(len(dicts['words2idx']), 300, weights=[embedding_matrix], input_length=maxLen, trainable=False))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Bidirectional(LSTM(100,return_sequences=True)))
model.add(Dropout(0.25))
model.add(TimeDistributed(Dense(n_classes, activation='softmax')))
model.compile('rmsprop', 'categorical_crossentropy')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 300)           171600    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 50, 200)           320800    
_________________________________________________________________
bidirectional_4 (Bidirection (None, 50, 200)           240800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 50, 200)           0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, 50, 127)           25527     
Total params: 758,727
Trainable params: 587,127
Non-trainable params: 171,600
_________________________________________________________________


In [0]:
import progressbar
n_epochs = 30

for i in range(n_epochs):
    print("Training epoch {}".format(i))
    
    bar = progressbar.ProgressBar(max_value=len(padded_docs))
    for n_batch, sent in bar(enumerate(padded_docs)):
        label = train_label[n_batch]
        label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
        # Make labels one hot
        label = np.eye(n_classes)[label][np.newaxis,:]
        # View each sentence as a batch
        sent = sent[np.newaxis,:]
        
        if sent.shape[1] > 1: #ignore 1 word sentences
            model.train_on_batch(sent, label)

N/A% (0 of 4978) |                       | Elapsed Time: 0:00:00 ETA:  --:--:--

Training epoch 0


100% (4978 of 4978) |####################| Elapsed Time: 0:08:05 Time:  0:08:05
  0% (1 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:09:44

Training epoch 1


100% (4978 of 4978) |####################| Elapsed Time: 0:09:00 Time:  0:09:00
  0% (1 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:11:16

...

100% (4978 of 4978) |####################| Elapsed Time: 0:09:25 Time:  0:09:25
  0% (1 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:10:36

Training epoch 28


100% (4978 of 4978) |####################| Elapsed Time: 0:09:23 Time:  0:09:23
  0% (1 of 4978) |                       | Elapsed Time: 0:00:00 ETA:   0:10:03

Training epoch 29


100% (4978 of 4978) |####################| Elapsed Time: 0:09:24 Time:  0:09:24


In [0]:
from metrics.accuracy import conlleval

labels_pred_val = []

from keras.preprocessing.sequence import pad_sequences
padded_docs = pad_sequences(val_x, maxlen=maxLen, padding='post')

bar = progressbar.ProgressBar(max_value=len(padded_docs))
for n_batch, sent in bar(enumerate(padded_docs)):
    label = val_label[n_batch]
    label = np.pad(label, (0, maxLen - len(label)%maxLen), 'constant')
    label = np.eye(n_classes)[label][np.newaxis,:]
    sent = sent[np.newaxis,:]

    pred = model.predict_on_batch(sent)
    pred = np.argmax(pred,-1)[0]
    labels_pred_val.append(pred)

labels_pred_val = [ list(map(lambda x: idx2la[x], y)) \
                                    for y in labels_pred_val]
con_dict = conlleval(labels_pred_val, labels_val, 
                            words_val, 'measure.txt')

print('Precision = {}, Recall = {}, F1 = {}'.format(
            con_dict['r'], con_dict['p'], con_dict['f1']))


100% (893 of 893) |######################| Elapsed Time: 0:00:10 Time:  0:00:10


Precision = 95.28, Recall = 95.48, F1 = 95.38
